In [2]:
# imports
from PIL import Image, ImageChops, ImageOps
from PIL import Image, ImageFont, ImageDraw 
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras


In [36]:
img_empty = Image.open("./../Data/Stereo_conveyor_without_occlusions_undistorted/left/1585434280_502471924_Left.png")
img_empty = ImageOps.grayscale(img_empty)
folder = "./../Data/Stereo_conveyor_without_occlusions_undistorted/left"
cropped_images = []
images = []
class_names = ['Book', 'Box', 'Cup']
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (0, 0, 0)
thickness = 3
model = keras.models.load_model("./../Models/CNN_1.kerasave")

for filename in os.listdir(folder):
    img = Image.open(os.path.join(folder, filename))
    img_gray = ImageOps.grayscale(img)
    
    
    # finding differences
    diff = ImageChops.difference(img_empty, img_gray)
    diff_np = np.array(diff)
    
    ret,thresh1 = cv2.threshold(diff_np,20,255,cv2.THRESH_BINARY)
    # Morphological operations to delete unwanted small areas
    kernel = np.ones((10,10), np.uint8)
    img_erosion = cv2.erode(thresh1, kernel, iterations = 1)
    img_dilation = cv2.dilate(img_erosion, kernel, iterations = 1)
    
    #cut
    img_dilation[0:720, 0:330] = 0
    img_dilation[0:720, 1100:1280] = 0
    img_dilation[600:720, 0:1280] = 0

    # create rectangle around the object
    img = np.array(img)
    #img = keras.preprocessing.image.img_to_array(img)
    coords = cv2.findNonZero(img_dilation)
    x,y,w,h = cv2.boundingRect(coords)
    if (img is not None) and (w < 400) and (w > 50) and (h < 400) and (h > 50):
        rec = cv2.rectangle(img, (x-40,y-25), (x+w+25,y+h+25), (0,255,0), 4)
        x_start = x - 40
        x_end = x + w +20
        y_start = y-20
        y_end = y + h+20
        if x_start < 0:
            x_start = 0
        if x_end > img_gray.size[0]:
            x_end = img_gray.size[0]
        if y_start < 0: 
            y_start = 0
        if y_end > img_gray.size[1]:
            y_end =  img_gray.size[1]

    
        img_crop = img[y_start:y_end,x_start:x_end]
  
        img_crop = cv2.resize(img_crop,(180,180), interpolation = cv2.INTER_AREA)
    
        img_array = tf.expand_dims(img_crop, 0)
    
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        class_predict = class_names[np.argmax(score)]
        text = "Detected object: " + class_predict
    else:
        rec = img
        text = "Detected object: None"
    
    
    if (rec is not None):
        image = cv2.putText(rec, text, org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image) 


In [38]:
#save as video

out = cv2.VideoWriter('./../Output/Classifier.avi',cv2.VideoWriter_fourcc(*'DIVX') , 15, (images[1].shape[1], images[1].shape[0]))

for i in range(len(images)):
    out.write(images[i])

out.release()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7012705b-60eb-4206-a7ec-0352216dd398' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>